# Actualiza horarios de clases desde grace

## Importes y utilidades

In [1]:
import requests
import urllib.request
import json
import os
from bs4 import BeautifulSoup

In [2]:
def getHTML(url):
    with urllib.request.urlopen(url) as u:
        html=u.read().decode("utf-8","ignore")
    return html

In [3]:
#TODO creo que esto no es levenshtein xd
def levenshtein_ratio(a,b):
    a=a.lower().strip()
    b=b.lower().strip()
    edits_away=0
    for index,letter in enumerate(a):
        try:
            if letter!=b[index]:edits_away+=1
        except:edits_away+=1
    ratio=((len(a)+len(b)) - edits_away) / (len(a)+len(b))
    return ratio

In [4]:
def replace_latin_chars(str):
    translate = {"Á": "A", "É": "E", "Í": "I", "Ó": "O", "Ú": "U", "Ñ": "N", "Ü": "U","á": "a", "é": "e", "í": "i", "ó": "o", "ú": "u", "ñ": "n", "ü": "u"}
    for original,replacement in translate.items():
        str.replace(original,replacement)
    return str

In [5]:
def fix_url(str):
    nombre = str.strip();
    nombre = replace_latin_chars(nombre);
    nombre = nombre.replace("/[^a-zA-Z0-9\s]/", "");
    nombre = nombre.replace("(", "");
    nombre = nombre.replace(")", "");
    nombre = nombre.replace(" -", "-");
    nombre = nombre.replace("- ", "-");
    nombre = nombre.replace("\n", "");
    nombre = nombre.replace("\r", "");
    nombre = nombre.replace("  ", " ");
    nombre = nombre.replace("\\", "");
    nombre = nombre.replace(" ", "-");
    return nombre;

## Scrap grace

In [6]:
#Regresa periodo y lista de nombre de clases dado el url del dropdown de grace
def preliminaryInfo(url):
    html=getHTML(url)
    periodo=(html.split("período")[1].split("</h3>")[0]).strip()
    clases=[i.replace('</option>','').replace('\n','') for i in html.split("<option>")[1:-1]]  
    return periodo,clases

In [7]:
#Dado el html de form_url regresa diccionario de la forma {"grupo":{diccionario con info de grupo}} y un booleado laboratorio
def getClaseInfo(html):
    if not html:return None
    grupos={}
    teoria=False
    laboratorio=False
    info=["depto","clave","grupo","TL","nombre","prof","cred","horario","dias","salon","campus","comentarios"] #datos por extraer
    soup=BeautifulSoup(html, 'html.parser')
    tabla=soup.find_all('table')[2] #la tabla con la info es la 3ra en el html
    for renglon in tabla.find_all('tr')[1:]: #saltar renglon con cabecera
        grupo={i:None for i in info}
        info_index=0 #indice para navegar info[]
        for columna in renglon.find_all('td'):
            grupo[info[info_index]]=columna.string.strip()
            if info[info_index]=="dias":
                grupo[info[info_index]]=grupo[info[info_index]].split()
            info_index+=1
        grupos[grupo["grupo"]]=grupo
    return grupos

In [8]:
#Returns dict w clave,nombre,grupos
def formatClases(clases):
    formated=[] 
    for c in clases:
        grupos=[]
        for clave,data in c.items():
            nombre=data['depto']+'-'+data['clave']+'-'+data['nombre']
            nombre=nombre if data['TL']=='T' else nombre+'-LAB'
            d={'grupo':data['grupo'],'nombre':nombre,'profesor':data['prof'],'creditos':data['cred'],'horario':data['horario'],'dias':data['dias'],'salon':data['salon'],'campus':data['campus']}
            grupos.append(d)
        temp={'nombre':nombre,'clave':data['clave'],'grupos':grupos}
        formated.append(temp)
    return formated

In [9]:
s="1920"
dropDownUrl="https://serviciosweb.itam.mx/EDSUP/BWZKSENP.P_Horarios1?s="+s
formPostUrl="https://serviciosweb.itam.mx/EDSUP/BWZKSENP.P_Horarios2"

In [10]:
periodo,listaClases=preliminaryInfo(dropDownUrl)
periodo,len(listaClases)

('OTOÑO 2021 LICENCIATURA', 413)

In [11]:
clases=[getClaseInfo(requests.post(url=formPostUrl,data={"s":s,"txt_materia":nombre}).text) for nombre in listaClases]

In [12]:
formatedClases=formatClases(clases)

In [13]:
formatedClases

[{'nombre': 'ACT-11300-CALCULO ACTUARIAL I',
  'clave': '11300',
  'grupos': [{'grupo': '001',
    'nombre': 'ACT-11300-CALCULO ACTUARIAL I',
    'profesor': 'SERGIO GARCIA ALQUICIRA',
    'creditos': '6',
    'horario': '19:00-20:30',
    'dias': ['MA', 'JU'],
    'salon': 'RH103',
    'campus': 'RIO HONDO'},
   {'grupo': '002',
    'nombre': 'ACT-11300-CALCULO ACTUARIAL I',
    'profesor': 'PEDRO DANIEL TREJO SOLIS',
    'creditos': '6',
    'horario': '19:00-20:30',
    'dias': ['LU', 'MI'],
    'salon': 'RH302',
    'campus': 'RIO HONDO'},
   {'grupo': '003',
    'nombre': 'ACT-11300-CALCULO ACTUARIAL I',
    'profesor': 'FERNANDO PEREZ RIOS',
    'creditos': '6',
    'horario': '07:00-08:30',
    'dias': ['LU', 'MI'],
    'salon': 'RH301',
    'campus': 'RIO HONDO'}]},
 {'nombre': 'ACT-11301-CALCULO ACTUARIAL II',
  'clave': '11301',
  'grupos': [{'grupo': '001',
    'nombre': 'ACT-11301-CALCULO ACTUARIAL II',
    'profesor': 'SERGIO GARCIA ALQUICIRA',
    'creditos': '6',
    'ho

In [16]:
with open("clases.js","w+") as f:
    f.write("clases="+json.dumps(clases))

In [18]:
#Use saved clases
with open("clases.js","r") as f:
    clases=json.loads(f.read().split('clases=')[1])

## Scrap MisProfes.com

In [14]:
misProfesUrl='https://www.misprofesores.com/escuelas/ITAM-Instituto-Tecnologico-Autonomo-de-Mexico_1003'
misProfesHTML=getHTML(misProfesUrl)
def profUrl(nombre,apellido,id):
    return "https://www.misprofesores.com/profesores/"+fix_url(nombre)+"-"+fix_url(apellido)+"_"+id;

In [15]:
start=misProfesHTML.index('var dataSet = ')+len('var dataSet = ')
tempData=json.loads(misProfesHTML[start:misProfesHTML.index('];',start)+1])
misProfesData={p['n']+' '+p['a']:{'c':p['c'],'m':p['m'],'i':p['i'],'misProfesNom':p['n']+' '+p['a'],'link':profUrl(p['n'],p['a'],p['i'])} for p in tempData}

## Match profesores

In [16]:
profesores=list(set([grupo['prof'] for clase in clases for grupo in clase.values() if len(grupo['prof'].strip())>1]))

In [17]:
matchedData={p:misProfesData[m] for m in misProfesData.keys() for p in profesores if levenshtein_ratio(p,m)>0.90}

In [18]:
matchedData

{'GERARDO ACOSTA GARCIA': {'c': '6.3333',
  'm': '6',
  'i': '105240',
  'misProfesNom': 'Gerardo Acosta Garcia',
  'link': 'https://www.misprofesores.com/profesores/Gerardo-Acosta-Garcia_105240'},
 'MARIA DE LAS MERCEDES ADAMUZ PEÑA': {'c': '6.5833',
  'm': '24',
  'i': '10746',
  'misProfesNom': 'María de las Mercedes Adamuz Peña',
  'link': 'https://www.misprofesores.com/profesores/María-de-las-Mercedes-Adamuz-Peña_10746'},
 'ARTURO ALBERTO AGUILAR ESTEVA': {'c': '8.4167',
  'm': '12',
  'i': '58325',
  'misProfesNom': 'Arturo Alberto Aguilar Esteva',
  'link': 'https://www.misprofesores.com/profesores/Arturo-Alberto-Aguilar-Esteva_58325'},
 'JUAN CARLOS AGUILAR VILLEGAS': {'c': '7.1875',
  'm': '48',
  'i': '10711',
  'misProfesNom': 'Juan Carlos Aguilar Villegas',
  'link': 'https://www.misprofesores.com/profesores/Juan-Carlos-Aguilar-Villegas_10711'},
 'CLAUDIA ALVAREZ ESTRADA': {'c': '9.5294',
  'm': '17',
  'i': '67068',
  'misProfesNom': 'Claudia Álvarez Estrada',
  'link': 'h

## Write data

In [19]:
links={p:d['link'] for p,d in matchedData.items()}
ratings={p:[float(d['c']),float(d['m'])] for p,d in matchedData.items() if d['c']}
listaClases=[]
for c in clases:
    t=list(c.values())[0]
    listaClases.append(t['depto']+'-'+t['clave']+'-'+t['nombre'])

In [20]:
clases

[{'001': {'depto': 'ACT',
   'clave': '11300',
   'grupo': '001',
   'TL': 'T',
   'nombre': 'CALCULO ACTUARIAL I',
   'prof': 'SERGIO GARCIA ALQUICIRA',
   'cred': '6',
   'horario': '19:00-20:30',
   'dias': ['MA', 'JU'],
   'salon': 'RH103',
   'campus': 'RIO HONDO',
   'comentarios': ''},
  '002': {'depto': 'ACT',
   'clave': '11300',
   'grupo': '002',
   'TL': 'T',
   'nombre': 'CALCULO ACTUARIAL I',
   'prof': 'PEDRO DANIEL TREJO SOLIS',
   'cred': '6',
   'horario': '19:00-20:30',
   'dias': ['LU', 'MI'],
   'salon': 'RH302',
   'campus': 'RIO HONDO',
   'comentarios': ''},
  '003': {'depto': 'ACT',
   'clave': '11300',
   'grupo': '003',
   'TL': 'T',
   'nombre': 'CALCULO ACTUARIAL I',
   'prof': 'FERNANDO PEREZ RIOS',
   'cred': '6',
   'horario': '07:00-08:30',
   'dias': ['LU', 'MI'],
   'salon': 'RH301',
   'campus': 'RIO HONDO',
   'comentarios': ''}},
 {'001': {'depto': 'ACT',
   'clave': '11301',
   'grupo': '001',
   'TL': 'T',
   'nombre': 'CALCULO ACTUARIAL II',
   

In [21]:
#Remplazar datos
with open("index.html","r",encoding="utf8") as f:
    oldHTML=f.read()
with open("index.html","w",encoding="utf8") as f:
    f.write(oldHTML.replace("<!--d-->",periodo));

with open("generar_horarios.js","r",encoding="utf8") as f:
    old=f.read()
essential=old[:old.index("/**DATA*/")]
os.remove("backup.js")
os.rename("generar_horarios.js","backup.js")
with open("generar_horarios.js","w+",encoding="utf8") as f:
    f.write(essential)
    f.write("\n /**DATA*/")
    f.write("\nvar lista_de_todas_clases="+json.dumps(listaClases))
    f.write("\nvar ratings="+json.dumps(ratings))
    f.write("\nvar clases="+json.dumps(formatedClases))
    f.write("\nvar links="+json.dumps(links))